In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict

In [2]:
import sys
import os
def add_sys_path(p):
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

add_sys_path('..')

In [3]:
import evaluate

In [4]:
import data_split

In [5]:
from evaluate import read_dataset

In [6]:
import json

In [29]:
from tqdm.auto import tqdm, trange

# Чтение и сплит данных

In [9]:
n_ds = read_dataset('../data/training_data/training_nouns.tsv',  lambda x: json.loads(x))

In [10]:
len(n_ds)

25376

In [11]:
train_share = 0.8
dev_share = 0.02
test1_share = 0.02
test2_share = 0.02
hid_share = 0.14

assert train_share + dev_share + test1_share + test2_share + hid_share == 1

In [12]:
from data_split import hash_float

In [13]:
train, dev, test1, test2, hid = {}, {}, {} , {}, {}

for k, v in n_ds.items():
    h = hash_float(k)
    if h <= train_share:
        train[k] = v
    elif h <= train_share + dev_share:
        dev[k] = v
    elif h <= train_share + dev_share + test1_share:
        test1[k] = v
    elif h <= train_share + dev_share + test1_share + test2_share:
        test2[k] = v
    else:
        hid[k] = v

# Применение бейзлайна

In [14]:
import my_knn
from importlib import reload
reload(my_knn)

<module 'my_knn' from 'C:\\Users\\ddale\\YandexDisk\\code\\NLP\\taxonomy-enrichment\\experiments\\my_knn.py'>

In [15]:
import gensim
# this is araneum 2018 ft model from rusvectores
ft = gensim.models.fasttext.FastTextKeyedVectors.load(
    'C:/Users/ddale/Downloads/NLP/rusvectores/model.model'
)

In [16]:
embedder = my_knn.SentenceEmbedder(ft=ft, n=300)

In [17]:
embedder('привет как дела').shape

(300,)

In [21]:
import xmltodict
with open('../data/ruwordnet/synsets.N.xml', 'r', encoding='utf-8') as f:
    synsets_n_raw = xmltodict.parse(f.read(), process_namespaces=True)

In [22]:
with open('../data/ruwordnet/synset_relations.N.xml', 'r', encoding='utf-8') as f:
    rel_n_raw = xmltodict.parse(f.read(), process_namespaces=True)

In [23]:
syns_storage = my_knn.SynsetStorage.construct(synsets_n_raw)

29296 119273


In [24]:
rel_df = my_knn.make_rel_df(rel_n_raw, syns_storage.id2synset)

In [25]:
rel_df.head()

,@parent_id,@child_id,@name,parent,child
0,147272-N,147272-A,POS-synonymy,КРЕСТНЫЙ РОДИТЕЛЬ,None
1,147272-N,4544-N,hypernym,КРЕСТНЫЙ РОДИТЕЛЬ,ХРИСТИАНИН
2,147272-N,126551-N,hyponym,КРЕСТНЫЙ РОДИТЕЛЬ,КРЕСТНЫЙ ОТЕЦ
3,147272-N,2201-N,domain,КРЕСТНЫЙ РОДИТЕЛЬ,РЕЛИГИЯ
4,147272-N,1150-N,domain,КРЕСТНЫЙ РОДИТЕЛЬ,ХРИСТИАНСТВО


In [26]:
rel_storage = my_knn.RelationStorage() # todo: add forbidden_id
rel_storage.construct_relations(rel_df)


28118
555
5
37249


In [27]:
len(syns_storage.texts_long)

119273

In [30]:
vecs = np.stack([embedder(t) for t in tqdm(syns_storage.texts_long) ])

In [31]:
from sklearn.neighbors import KDTree
tree = KDTree(vecs)

In [33]:
hypos = my_knn.hypotheses_knn('брюки', index=tree, text2vec=embedder, synset_storage=syns_storage, rel_storage=rel_storage)
hypos

['1256-N', '8218-N', '109873-N', '108194-N', '4024-N', '108612-N', '108091-N']

In [34]:
for id in hypos:
    print(syns_storage.id2synset[id]['@ruthes_name'])

ОДЕЖДА
ИЗДЕЛИЕ ЛЕГКОЙ ПРОМЫШЛЕННОСТИ
ИЗДЕЛИЕ
ШТАНЫ, БРЮКИ
ВЕРХНЯЯ ОДЕЖДА
КОСТЮМ (ОДЕЖДА)
ЖЕНСКАЯ ОДЕЖДА


In [35]:
# проверка, что сравнение само с собой всегда получает скор 1
dev_pred = {k: [v0 for val in vals for v0 in val] for k, vals in dev.items()}
mean_ap, mean_rr = evaluate.get_score(dev, dev_pred, k=10)
print(mean_ap, mean_rr)

0.9996719160104987 1.0


In [36]:
#texts = ttest_test1.one_text.drop_duplicates()
texts = sorted(dev.keys())
small_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=tree, text2vec=embedder, synset_storage=syns_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
    )  
    for txt in tqdm(texts)
}


In [37]:
mean_ap, mean_rr = evaluate.get_score(dev, small_hypos, k=10)
print(mean_ap, mean_rr)

0.5949077979835851 0.6197334708161476


# Оценка применения

In [38]:
public_test_verbs = pd.read_csv('../data/public_test/nouns_public.tsv', header=None)
public_test_verbs.columns = ['text']

In [39]:
public_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=tree, text2vec=embedder, synset_storage=syns_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
    )  
    for txt in tqdm(public_test_verbs.text)
}

In [40]:
sub = my_knn.dict2submission(public_test_hypos, syns_storage.id2synset)

In [41]:
sub

,noun,result,result_text
0,АБДОМИНОПЛАСТИКА,106536-N,МАТЕРИАЛ ДЛЯ ИЗГОТОВЛЕНИЯ
1,АБДОМИНОПЛАСТИКА,919-N,СИНТЕТИЧЕСКИЕ МАТЕРИАЛЫ
2,АБДОМИНОПЛАСТИКА,132393-N,ПРОДУКЦИЯ ПРОИЗВОДСТВЕННО-ТЕХНИЧЕСКОГО НАЗНАЧЕНИЯ
3,АБДОМИНОПЛАСТИКА,540-N,ИСКУССТВО
4,АБДОМИНОПЛАСТИКА,118950-N,"КРАСОТА, КРАСА"
...,...,...,...
7615,ЯТАГАН,106554-N,"ПРЕДМЕТ, ВЕЩЬ"
7616,ЯТАГАН,153018-N,РУБЯЩЕЕ ОРУЖИЕ
7617,ЯТАГАН,107709-N,ПАЛКА (ПРЕДМЕТ)
7618,ЯТАГАН,153561-N,ПИРОТЕХНИЧЕСКИЕ ИЗДЕЛИЯ


In [42]:
sub.to_csv('results/nouns_v0_strange.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АБДОМИНОПЛАСТИКА,106536-N,МАТЕРИАЛ ДЛЯ ИЗГОТОВЛЕНИЯ
1,АБДОМИНОПЛАСТИКА,919-N,СИНТЕТИЧЕСКИЕ МАТЕРИАЛЫ
2,АБДОМИНОПЛАСТИКА,132393-N,ПРОДУКЦИЯ ПРОИЗВОДСТВЕННО-ТЕХНИЧЕСКОГО НАЗНАЧЕНИЯ
3,АБДОМИНОПЛАСТИКА,540-N,ИСКУССТВО
4,АБДОМИНОПЛАСТИКА,118950-N,"КРАСОТА, КРАСА"
5,АБДОМИНОПЛАСТИКА,511-N,НОСИТЕЛЬ ИНФОРМАЦИИ
6,АБДОМИНОПЛАСТИКА,920-N,ПЛАСТМАССА
7,АБДОМИНОПЛАСТИКА,4241-N,МАГНИТНЫЙ НОСИТЕЛЬ
8,АБДОМИНОПЛАСТИКА,110935-N,ПИСЬМЕННЫЕ ПРИНАДЛЕЖНОСТИ
9,АБДОМИНОПЛАСТИКА,144594-N,КОСМЕТИЧЕСКАЯ ОПЕРАЦИЯ


# Финальная заливка

In [48]:
private_test_verbs = pd.read_csv('../data/private_test/nouns_private.tsv', header=None)
private_test_verbs.columns = ['text']

In [44]:
private_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=tree, text2vec=embedder, synset_storage=syns_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
    )  
    for txt in tqdm(private_test_verbs.text)
}

In [45]:
sub = my_knn.dict2submission(private_test_hypos, syns_storage.id2synset)

In [46]:
sub

,noun,result,result_text
0,АБСЕНТЕИЗМ,119563-N,МИРОВОЗЗРЕНИЕ
1,АБСЕНТЕИЗМ,132973-N,ФИЛОСОФСКОЕ УЧЕНИЕ
2,АБСЕНТЕИЗМ,121514-N,УЧЕНИЕ (СОВОКУПНОСТЬ ТЕОРЕТИЧ. ПОЛОЖЕНИЙ)
3,АБСЕНТЕИЗМ,106470-N,"УБЕЖДЕНИЯ, ВЗГЛЯДЫ"
4,АБСЕНТЕИЗМ,2201-N,РЕЛИГИЯ
...,...,...,...
15245,ЯШМА,106554-N,"ПРЕДМЕТ, ВЕЩЬ"
15246,ЯШМА,109444-N,УКРАСИТЬ
15247,ЯШМА,820-N,ВЕЩЕСТВО
15248,ЯШМА,143683-N,ПРИРОДНОЕ МИНЕРАЛЬНОЕ ОБРАЗОВАНИЕ


In [47]:
sub.to_csv('results/private_nouns_v0_strange.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АБСЕНТЕИЗМ,119563-N,МИРОВОЗЗРЕНИЕ
1,АБСЕНТЕИЗМ,132973-N,ФИЛОСОФСКОЕ УЧЕНИЕ
2,АБСЕНТЕИЗМ,121514-N,УЧЕНИЕ (СОВОКУПНОСТЬ ТЕОРЕТИЧ. ПОЛОЖЕНИЙ)
3,АБСЕНТЕИЗМ,106470-N,"УБЕЖДЕНИЯ, ВЗГЛЯДЫ"
4,АБСЕНТЕИЗМ,2201-N,РЕЛИГИЯ
5,АБСЕНТЕИЗМ,108333-N,СТИЛЬ В ИСКУССТВЕ
6,АБСЕНТЕИЗМ,124135-N,ИДЕОЛОГИЯ
7,АБСЕНТЕИЗМ,111965-N,ИДЕАЛИЗМ
8,АБСЕНТЕИЗМ,121164-N,ИЗВРАЩЕНИЕ (ПРОТИВОЕСТЕСТВЕННОЕ ПОВЕДЕНИЕ)
9,АБСЕНТЕИЗМ,6639-N,ПОЛИТИЧЕСКОЕ ТЕЧЕНИЕ
